In [39]:
# To import configurations from config.ini files
import configparser
# For dataframe processes
import pandas as pd
import numpy as np

# For vizualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix

# To create a quick model to look at Feature Importances
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

# To save csv's with current date information
import datetime as dt

In [40]:
# import and read my config.ini file
config = configparser.ConfigParser()
config.read("../src/config.ini")

['../src/config.ini']

In [41]:
# Import my output path for saving data
output = config['paths']['data_path']

# Main Model

In [42]:
train_data = pd.read_pickle(config['paths']['train_data_clean'])
train_lbls = pd.read_pickle(config['paths']['train_labels_clean'])
test_data = pd.read_pickle(config['paths']['test_data_clean'])

# train_data = pd.read_csv(config['paths']['train_data_clean'])
# train_lbls = pd.read_csv(config['paths']['train_labels_clean'])
# test_data = pd.read_csv(config['paths']['test_data_clean'])

# Import submission format
sub_form = pd.read_csv(config['paths']['sub_form'])

In [43]:
train_data

,amount_tsh,gps_height,longitude,latitude,region_code,population,public_meeting,permit,x0_0,x0_Danida,...,x15_shallow well,x15_spring,x15_unknown,x16_cattle trough,x16_communal standpipe,x16_communal standpipe multiple,x16_dam,x16_hand pump,x16_improved spring,x16_other
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,1390,34.93750,-9.859375,11,109,True,False,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8776,0.0,1399,34.68750,-2.148438,20,280,False,True,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34310,25.0,686,37.46875,-3.822266,21,250,True,True,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
67743,0.0,263,38.50000,-11.156250,90,58,True,True,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19728,0.0,0,31.12500,-1.825195,18,0,True,True,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60739,10.0,1210,37.15625,-3.253906,3,125,True,True,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
27263,4700.0,1212,35.25000,-9.070312,11,56,True,True,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
37057,0.0,0,34.03125,-8.750000,12,0,True,False,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [44]:
# train_data.index = train_data.id 
# test_data.index = test_data.id
# train_lbls = train_lbls.id 
# train_data.drop(columns='id', inplace=True)
# test_data.drop(columns='id', inplace=True)
# train_lbls.drop(columns='id', inplace=True)


In [45]:
rf = RandomForestClassifier(random_state=42)

In [46]:
rf.fit(train_data, train_lbls)

C:\Users\Cristian\AppData\Local\Temp\ipykernel_23340\886976610.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_data, train_lbls)


RandomForestClassifier(random_state=42)

In [47]:
preds = rf.predict(test_data)
sub_form['status_group'] = preds

In [48]:
current_time = dt.datetime.now().strftime("%d%m%Y_%I%M%p")
sub_form.to_csv(output+ 'my_submission'+current_time+'.csv',index=False)

In [49]:
sub_form

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,non functional
14846,18990,functional
14847,28749,functional
14848,33492,functional
